In [1]:
from functools import reduce
import numpy as np
import json
from tqdm import tqdm

In [2]:
with open("./data_inputs/day18_input.txt") as f:
    input = f.read().split("\n")

In [3]:
input = [json.loads(line) for line in input]

In [4]:
def pair_2_leafs(pair):
    
    def _recursive_pair_2_leafs(pair, level):
        if isinstance(pair[0], list):
            _recursive_pair_2_leafs(pair[0], level+1)
        else:
            leafs.append([level, pair[0]])

        if isinstance(pair[1], list):
            _recursive_pair_2_leafs(pair[1], level+1)
        else:
            leafs.append([level, pair[1]])

    leafs = []
    _recursive_pair_2_leafs(pair, 0)
    return leafs

In [5]:
def leafs_2_pair(leafs):
    depth = max(leafs, key=lambda x: x[0])[0]
    pair = leafs.copy()
    for l in range(depth, -1, -1):   # level
        pair_copy = []
        pair_appended = False
        for i, p in enumerate(pair):
            if pair_appended:
                pair_appended = False
                continue

            if p[0] == l:
                pair_copy.append([l-1, [pair[i][1], pair[i+1][1]]])
                pair_appended = True
            else:
                pair_copy.append(p)

        pair = pair_copy.copy()
        #print("level:", l, pair)
    
    return pair[0][1]

In [6]:
def leafs_explode(leafs):
    il4 = None
    for i, l in enumerate(leafs):
        if l[0] == 4:
            il4 = i
            break
    
    if il4 != None:
        if il4 > 0:
            leafs[il4-1][1] += leafs[il4][1]
        
        leafs[il4] = [3, 0]

        if (il4 + 1) < (len(leafs) - 1):
            leafs[il4+2][1] += leafs[il4+1][1]
        
        leafs.pop(il4+1)

        leafs_explode(leafs)

    else:
        return None

In [7]:
def leafs_split(leafs):
    for i, l in enumerate(leafs):
        if l[1] > 9:
            new_leaf = [[l[0]+1, l[1]//2], [l[0]+1, int(l[1]/2 + 0.5)]]
            leafs[i] = new_leaf[0]
            leafs.insert(i+1, new_leaf[1])
            break


In [8]:
def reduce_pair(pair):
    leafs = pair_2_leafs(pair)

    #print(pair, "-->", leafs)
    i = 0
    while((max(leafs, key=lambda x: x[0])[0] == 4) or (max(leafs, key=lambda x: x[1])[1] > 9)):
        leafs_explode(leafs)
        #print(leafs_2_pair(leafs))
        leafs_split(leafs)
        #print(leafs_2_pair(leafs),"\n")
        i += 1
        if i > 1000:
            print("while break i > 1000")
            break

    return leafs_2_pair(leafs)

In [9]:
def pair_2_leafs_2_magnitude(pair):
    leafs = pair_2_leafs(pair)
    depth = max(leafs, key=lambda x: x[0])[0]
    pair = leafs.copy()
    for l in range(depth, -1, -1):   # level
        pair_copy = []
        pair_appended = False
        for i, p in enumerate(pair):
            if pair_appended:
                pair_appended = False
                continue

            if p[0] == l:
                pair_copy.append([l-1, pair[i][1]*3 + pair[i+1][1]*2])
                pair_appended = True
            else:
                pair_copy.append(p)

        pair = pair_copy.copy()
        #print("level:", l, pair)
    
    return pair[0][1]

In [10]:
# ----- PART 1 -----

sum_res = reduce(lambda x, y: reduce_pair([x, y]), input)
magnitude = pair_2_leafs_2_magnitude(sum_res)

print("Result1:", magnitude)

Result1: 3892


In [11]:
# ----- PART 2 -----

max_magnitude = 0

for num in tqdm(input):
    for other_num in input:
        if other_num == num:
            continue
        magnitude = pair_2_leafs_2_magnitude(reduce_pair([num, other_num]))
        if magnitude > max_magnitude:
            max_magnitude = magnitude

print("Result2:", max_magnitude)


100%|██████████| 100/100 [00:01<00:00, 80.50it/s]

Result2: 4909
